In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier

In [2]:
dataset_home = "./amz"

In [16]:
train = pd.read_csv(f"{dataset_home}/train.csv", names=["target", "title", "data"])
test = pd.read_csv(f"{dataset_home}/test.csv", names=["target", "title", "data"])

In [17]:
train = train.drop(columns=['title'])
test = test.drop(columns=['title'])

In [18]:
train.head()

,target,data
0,3,Gave this to my dad for a gag gift after direc...
1,5,I hope a lot of people hear this cd. We need m...
2,5,I'm reading a lot of reviews saying that this ...
3,4,The music of Yasunori Misuda is without questi...
4,5,Probably the greatest soundtrack in history! U...


In [19]:
test.head()

,target,data
0,1,"This model may be ok for sedentary types, but ..."
1,4,This is a fast read filled with unexpected hum...
2,2,I bought one of these chargers..the instructio...
3,2,I was excited to find a book ostensibly about ...
4,2,"I am a big JVC fan, but I do not like this mod..."


In [20]:
# NEUTRAL = 0, NEG = 1, POS = 2
train.loc[(train.target == 2),'target'] = 1
train.loc[(train.target == 4),'target'] = 2
train.loc[(train.target == 5),'target'] = 2
train.loc[(train.target == 3),'target'] = 0

test.loc[(test.target == 2),'target'] = 1
test.loc[(test.target == 4),'target'] = 2
test.loc[(test.target == 5),'target'] = 2
test.loc[(test.target == 3),'target'] = 0

train_df = train.groupby('target').sample(n=40000, random_state=123).sample(frac=1)
train_df_index = train_df.index
train_df = train_df.reset_index(drop=True)
test_df = test.groupby('target').sample(n=10000, random_state=123).sample(frac=1).reset_index(drop=True)

In [22]:
frames = [train_df, test_df]
corpus = pd.concat(frames, ignore_index=True)

vectorizer = TfidfVectorizer(max_df=0.4, ngram_range=(1, 2))
tfidf = vectorizer.fit_transform(corpus['data'])

X_train = tfidf[:train_df.shape[0]]
Y_train = train_df['target'].values

X_test = tfidf[train_df.shape[0]:]
Y_test = test_df['target'].values

In [25]:
clf = MLPClassifier(random_state=123, max_iter=500, early_stopping=True)
clf.fit(X_train, Y_train)

/Users/abhiravg/dl-project/repo/venv/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=123, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [30]:
clf.score(X_test, Y_test)

0.6838